In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [2]:
#Load the train model, scaler , pickle and onehot
model = load_model('model.h5')

#load the encoder and scaler
with open("label_encoder_gender.pkl",'rb') as file:
    label_encoder_gender = pickle.load(file)

with open("onehot_encoder_geo.pkl","rb") as file:
    onehot_encoder_geo = pickle.load(file)

with open("scaler.pkl","rb") as file:
    scaler = pickle.load(file)

In [3]:
#Example input data

input_data ={
    'CreditScore' : 600 ,
    'Geography' : "Spain",	
     'Gender'	: "Male",
      'Age'	:  30,  
     'Tenure' : 3,
     'Balance' :40000, 
     'NumOfProducts' : 2,	
    'HasCrCard' : 2,
   'IsActiveMember' : 1,	
'EstimatedSalary' : 50000
}

In [4]:
#One hot encode Geography
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns = onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

C:\Users\Soham Lad\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [5]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Spain,Male,30,3,40000,2,2,1,50000


In [6]:
#Encoding Categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Spain,1,30,3,40000,2,2,1,50000


In [7]:
#Concatination of onehot encoded
input_df = pd.concat([input_df.drop('Geography',axis = 1),geo_encoded_df],axis = 1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,30,3,40000,2,2,1,50000,0.0,0.0,1.0


In [8]:
#Scailing the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.47154541,  0.90911166, -0.87031707, -0.69844549, -0.61206849,
         0.80510537,  2.84544692,  0.95214374, -0.84805047, -1.02020406,
        -0.57581067,  1.77464858]])

In [9]:
#Predict Churn
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 72ms/step


array([[0.0049645]], dtype=float32)

In [10]:
prediction_proba = prediction[0][0]

In [11]:
prediction_proba

0.0049645016

In [12]:
if prediction_proba > 0.5 :
    print("Customer is likely to Churn")
else :
    print("Customer is not likely to Churn")

Customer is not likely to Churn
